In [ ]:
!pip install wptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 32.8 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=31829d441961175c4bd1176c9800459cd1c2035c6f0355f19981bba0d0e3b7d6
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [ ]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install beautifulsoup4

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'

/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [ ]:
%cd drive
!ls

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping
'Copy of scrape_wikipedia_arabic_Siege_of_Beirut.ipynb'
 datasets_updated
 scrape_wikipedia_arabic_1982_Lebanon_War.ipynb
 scrape_wikipedia_arabic_Alexander_Haig.ipynb
 scrape_wikipedia_arabic_Assassination_of_Bachir_Gemayel.ipynb
 scrape_wikipedia_arabic_August_1982_Lebanese_presidential_election.ipynb
 scrape_wikipedia_arabic_Bachir_Gemayel.ipynb
 scrape_wikipedia_arabic_Israeli_Lebanese_conflict.ipynb
 scrape_wikipedia_arabic_Israeli_occupation_of_Southern_Lebanon.ipynb
 scrape_wikipedia_arabic_Operation_Mole_Cricket_19.ipynb
 scrape_wikipedia_arabic_Sabra_and_Shatila_massacre.ipynb
 scrape_wikipedia_arabic_September_1982_Lebanese_presidential_election.ipynb


In [ ]:
%cd MyDrive

[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [ ]:
!ls

'Copy of scrape_wikipedia_arabic_Siege_of_Beirut.ipynb'
 datasets_updated
 scrape_wikipedia_arabic_1982_Lebanon_War.ipynb
 scrape_wikipedia_arabic_Alexander_Haig.ipynb
 scrape_wikipedia_arabic_Assassination_of_Bachir_Gemayel.ipynb
 scrape_wikipedia_arabic_August_1982_Lebanese_presidential_election.ipynb
 scrape_wikipedia_arabic_Bachir_Gemayel.ipynb
 scrape_wikipedia_arabic_Israeli_Lebanese_conflict.ipynb
 scrape_wikipedia_arabic_Israeli_occupation_of_Southern_Lebanon.ipynb
 scrape_wikipedia_arabic_Operation_Mole_Cricket_19.ipynb
 scrape_wikipedia_arabic_Sabra_and_Shatila_massacre.ipynb
 scrape_wikipedia_arabic_September_1982_Lebanese_presidential_election.ipynb


In [ ]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [ ]:
page = wptools.page('Siege_of_Beirut')
page.get_parse()

en.wikipedia.org (parse) Siege_of_Beirut
Siege of Beirut (en) data
{
  image: <list(0)> 
  infobox: <dict(18)> conflict, partof, image, caption, date, plac...
  pageid: 40678669
  parsetree: <str(29041)> <root><template><title>pp</title><part><...
  requests: <list(1)> parse
  title: Siege of Beirut
  wikibase: Q2617818
  wikidata_url: https://www.wikidata.org/wiki/Q2617818
  wikitext: <str(20135)> {{pp|small=yes}}{{Short description|Israe...
}


In [ ]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Siege_of_Beirut",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Siege of Beirut')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1920 Nebi Musa riots
1929 Hebron massacre
1929 Palestine riots
1936–1939 Arab revolt in Palestine
1947–1948 civil war in Mandatory Palestine
1948 Arab–Israeli War
1948 Palestine war
1948 Palestinian expulsion and flight
1949 Armistice Agreements
1956 Rafah massacre
1968 Israeli raid on Lebanon
1978 South Lebanon conflict
1982 Lebanon War
1982–1983 Tyre headquarters bombings
1983 West Bank fainting epidemic
1990 Temple Mount killings
1991 Iraqi missile attacks against Israel
2000 Camp David Summit
2002 Arab League summit
2004 Beit Hanoun raid
2004 Israeli operation in Rafah
2004 Israeli operation in the northern Gaza Strip
2006 Gaza beach explosion
2006 Gaza cross-border raid
2006 Gaza–Israel conflict
2006 Israeli operation in Beit Hanoun
2006 Lebanon War
2006 shelling of Beit Hanoun
2008 Jerusalem bulldozer attack
2008 Jerusalem yeshiva attack
2008 breach of the Egypt–Gaza border
2009 Aftonbladet Israel controversy
2009 Al-Aqsa clashes
2010 Palestinian militancy campaign
2010–20

In [ ]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod

    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries

    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [ ]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [ ]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

                    df_political_party_en = pd.concat([df_political_party_en, pd.DataFrame({
                        'Name': [','.join(name if isinstance(name, list) else name)],
                        'Ideology': [','.join(ideology) if isinstance(ideology, list) else ideology],
                        'Political Position': [','.join(position) if isinstance(position, list) else position],
                        'Founder(s)': [founders],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)



                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']

                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'

                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')

                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'

                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')


                    df_political_party_ar = pd.concat([df_political_party_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar)],
                        'الإيديولوجيا': [','.join(ideology_ar)],
                        'الموقف السياسي': [','.join(position_ar)],
                        'المؤسس (المؤسسون)': [','.join(founders_ar)],
                    })], ignore_index=True)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1920 Nebi Musa riots
en.wikipedia.org (imageinfo) File:Jerusalem-nabi-moussa-april-1920.jpg
en.wikipedia.org (parse) 1929 Hebron massacre
en.wikipedia.org (imageinfo) File:1929 Hebron massacre infobox.jpg
en.wikipedia.org (parse) 1929 Palestine riots
en.wikipedia.org (imageinfo) File:Jews flee the Old City of Jerus...
en.wikipedia.org (parse) 1936–1939 Arab revolt in Palestine
en.wikipedia.org (imageinfo) File:Train hostages.jpg
en.wikipedia.org (parse) 1947–1948 civil war in Mandatory Palestine
en.wikipedia.org (imageinfo) File:PikiWiki Israel 20804 The Palmach.jpg
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1948 Palestinian expulsion and flight
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1956 Rafah massacre
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation L

['Al-Aqsa Martyrs Brigades']
-----------------
name:  ['Al-Aqsa Martyrs Brigades']
founders:  
ideology:  ['<!-- Palestinian nationalism and Anti-Zionism can be assumed', 'but DONT add any other ideologies unless you have a RECENT erence. -->', 'Plainlist', 'Anti-Zionism', 'Secularism', 'Palestinian nationalism']
position:  -
START
Cannot find the link: http://en.wikipedia.org/wiki/AlAqsa Martyrs Brigades
name:  Al-Aqsa Martyrs Brigades
name ar:  كتائب شهداء الأقصى (مترجمه)
name ar link:  -1
END
START
Cannot find the link: http://en.wikipedia.org/wiki/Palestinian nationalism and AntiZionism can be assumed
ideology:  <!-- Palestinian nationalism and Anti-Zionism can be assumed
ideology ar:  يمكن افتراض القومية الفلسطينية ومعاداة الصهيونية (مترجمه)
ideo ar link:  -1
Cannot find the link: http://en.wikipedia.org/wiki/but DONT add any other ideologies unless you have a RECENT erence
ideology:  but DONT add any other ideologies unless you have a RECENT erence. -->
ideology ar:  لكن لا تضيف 

en.wikipedia.org (parse) Al-Mourabitoun


[]
-----------------
name:  []
founders:  
ideology:  ['Nasserism', 'Arab nationalism', 'Arab socialism', 'Anti-imperialism', 'Pan-Arabism']
position:  ['Left-wing']
START
END
START
ideology:  Nasserism
ideology ar:  التيار الناصري 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AA%D9%8A%D8%A7%D8%B1_%D8%A7%D9%84%D9%86%D8%A7%D8%B5%D8%B1%D9%8A
ideology:  Arab nationalism
ideology ar:  قومية عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Arab socialism
ideology ar:  اشتراكية عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Anti-imperialism
ideology ar:  معاداة الاستعمارية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%A7%D8%B3%D8%AA%D8%B9%D9%85%D8%A7%D8%B1%D9%8A%D8%A9
Cannot find the link: http://en.wikipedia.org/wiki/PanArabism
ideology:  Pan-Arabism
ide

en.wikipedia.org (parse) Al-Tanzim


...

[[Saad Haddad]] (1977–1984)
['South Lebanon Army']
-----------------
name:  ['South Lebanon Army']
founders:  ,
ideology:  ['Multiconfessionalism', 'Factions:', 'Anti-Palestinianism', 'Lebanese nationalism', 'Maronite politics', 'Zionism', 'Anti-communism', 'Muslim-Christian Unity', 'Secularism']
position:  -
START
name:  South Lebanon Army
name ar:  جيش لبنان الجنوبي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AC%D9%8A%D8%B4_%D9%84%D8%A8%D9%86%D8%A7%D9%86_%D8%A7%D9%84%D8%AC%D9%86%D9%88%D8%A8%D9%8A
END
START
ideology:  Multiconfessionalism
ideology ar:  التعددية المذهبية (مترجمه)
ideo ar link:  -1
ideology:  Factions:
ideology ar:  الفصائل (مترجمه)
ideo ar link:  -1
Cannot find the link: http://en.wikipedia.org/wiki/AntiPalestinianism
ideology:  Anti-Palestinianism
ideology ar:  معاداة فلسطين (مترجمه)
ideo ar link:  -1
ideology:  Lebanese nationalism
ideology ar:  قومية لبنانية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%84%D8%A8%D9%86%D8%A7%D

en.wikipedia.org (parse) South Lebanon conflict (1985–2000)
en.wikipedia.org (imageinfo) File:South Lebanon conflict montage.png
en.wikipedia.org (parse) Soviet Union
en.wikipedia.org (parse) Strategic victory
en.wikipedia.org (parse) Suburb
en.wikipedia.org (parse) Sudan
en.wikipedia.org (parse) Suez Crisis
en.wikipedia.org (imageinfo) File:Tanks Destroyed Sinai.jpg
en.wikipedia.org (parse) Sykes–Picot Agreement
en.wikipedia.org (imageinfo) File:MPK1-426 Sykes Picot Agreement ...
en.wikipedia.org (parse) Syria
en.wikipedia.org (parse) Syrian Social Nationalist Party


Rabie Banat
[[Antoun Saadeh]]
['Syrian Social Nationalist Party']
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Rabie Banat,Antoun Saadeh,
ideology:  ['Antisemitism', 'Economic populism', 'Social nationalism', 'Fascism', 'Syrian nationalism', 'collapsible list', 'ubl', 'Anti-communism', 'Syrian irredentism']
position:  ['sometimes being labeled far-right.', 'In the past', 'the party or elements of its ideology or membership have been erred to as belonging to both the political left', 'and right', 'sometimes being labeled far-right.', 'Syncretic efn', 'and right']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
ideology:  Antisemitism
ideology ar:  معاداة السامية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%

en.wikipedia.org (parse) Syrians
en.wikipedia.org (parse) Taba Summit
en.wikipedia.org (parse) Tactical victory
en.wikipedia.org (parse) Tel Aviv–Jerusalem bus 405 suicide attack
en.wikipedia.org (imageinfo) File:405Memorial2016.JPG
en.wikipedia.org (parse) The Bhamdoun abduction operation (1982)
en.wikipedia.org (parse) The Clinton Parameters
en.wikipedia.org (parse) There was no such thing as Palestinians
en.wikipedia.org (parse) Time (magazine)
en.wikipedia.org (parse) Timeline of anti-Zionism
en.wikipedia.org (parse) Timeline of the Israeli–Palestinian conflict
en.wikipedia.org (parse) Tripartite Declaration of 1950
en.wikipedia.org (parse) Tripoli, Lebanon
en.wikipedia.org (parse) Trump peace plan
en.wikipedia.org (parse) Tunis
en.wikipedia.org (parse) Tunisia
en.wikipedia.org (parse) Tyre, Lebanon
en.wikipedia.org (parse) Tzipi Livni
en.wikipedia.org (imageinfo) File:Tzipi Livni (38130112).jpg
en.wikipedia.org (parse) U.S. Sixth Fleet
en.wikipedia.org (imageinfo) File:US Sixth Fl

skipping Wikipedia:Citation needed
skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox 1982 Lebanon War
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data Soviet Union
skipping Template:Country data Syria
skipping Template:Country data United States
skipping Template:Israeli–Palestinian conflict
skipping Template talk:Arab–Israeli armed engagements
skipping Template talk:Arab–Israeli conflict
skipping Template talk:Arab–Israeli conflict participants
skipping Template talk:Arab–Israeli diplomacy
skipping Template talk:Campaignbox 1982 Lebanon War
skipping Template talk:Israeli–Palestinian conflict
skipping Category:Articles with unsourced statements from April 2023
skipping Category:Articles with unsourced statements from August 2011
sk

In [ ]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Al-Aqsa Martyrs Brigades,<!-- Palestinian nationalism and Anti-Zionism ...,-,
1,,"Nasserism,Arab nationalism,Arab socialism,Anti...",Left-wing,
2,Al-Tanzim,"Anti-communism,Ultranationalism,Lebanese natio...",-,
3,Amal Movement,"Multiconfessionalism,Muslim–Christian unity,Le...",Centre-right,"Nabih Berri,Musa al-Sadr,Mostafa Chamran,Husse..."
4,Arab Higher Committee,"Arab nationalism,Anti-Zionism,Palestinian nati...",-,
5,Arab Liberation Army,"Pan-Arabism,Arab nationalism,Antisemitism,Anti...",-,
6,Arab Liberation Front,Baathism,-,
7,Arab Socialist Baath Party – Lebanon Region,"Neo-Baathism,ubl",-,
8,hy,"Revolutionary socialism,Armenian nationalism,M...",Left-wing to far-left,
9,Army of Islam,"Jihadism,Salafism,Sunni Islamism",-,


In [ ]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,كتائب شهداء الأقصى (مترجمه),يمكن افتراض القومية الفلسطينية ومعاداة الصهيون...,-,
1,,"التيار الناصري ,قومية عربية ,اشتراكية عربية ,م...",يسارية,
2,التنظيم (مترجمه),"معاداة الشيوعية ,عصبية قومية ,قومية لبنانية",-,
3,حركة أمل,"التعددية المذهبية (مترجمه),الوحدة الاسلامية ال...",يمين الوسط (مترجمه),"نبيه بري ,موسى الصدر (مترجمه),مصطفى تشمران ,حس..."
4,اللجنة العربية العليا,"قومية عربية ,معاداة الصهيونية ,وطنية فلسطينية",-,
5,جيش الإنقاذ,"القومية العربية (مترجمه),قومية عربية ,معاداة ا...",-,
6,جبهة التحرير العربية,بعثية,-,
7,حزب البعث العربي الاشتراكي (لبنان),"البعثية الجديدة (مترجمه),أوبل (مترجمه)",-,
8,هاى (مترجمه),"اشتراكية ثورية ,قومية أرمنية ,الماركسيةاللينين...",من اليسار إلى أقصى اليسار (مترجمه),
9,جيش الاسلام (مترجمه),"الجهادية (مترجمه),السلفية ,إسلاموية",-,


In [ ]:
datasets_dir = 'datasets_updated/Siege_of_Beirut/'

In [ ]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [ ]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [ ]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'

                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'

                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = pd.concat([df_politician_en, pd.DataFrame({
                        'Name': [name_inserted],
                        'Political Party': [','.join(party) if isinstance(party, list) else party],
                        'Nationality': [','.join(nationality) if isinstance(nationality, list) else nationality],
                        'Religion': [','.join(religion) if isinstance(religion, list) else religion],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)

                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)

                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'

                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'

                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'

                    df_politician_ar = pd.concat([df_politician_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                        'الحزب السياسي': [','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar],
                        'الجنسيه': [','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar],
                        'الدين': [','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar],
                    })], ignore_index=True)

                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1920 Nebi Musa riots
en.wikipedia.org (imageinfo) File:Jerusalem-nabi-moussa-april-1920.jpg
en.wikipedia.org (parse) 1929 Hebron massacre
en.wikipedia.org (imageinfo) File:1929 Hebron massacre infobox.jpg
en.wikipedia.org (parse) 1929 Palestine riots
en.wikipedia.org (imageinfo) File:Jews flee the Old City of Jerus...
en.wikipedia.org (parse) 1936–1939 Arab revolt in Palestine
en.wikipedia.org (imageinfo) File:Train hostages.jpg
en.wikipedia.org (parse) 1947–1948 civil war in Mandatory Palestine
en.wikipedia.org (imageinfo) File:PikiWiki Israel 20804 The Palmach.jpg
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1948 Palestinian expulsion and flight
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1956 Rafah massacre
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation L

['Abdel Aziz al-Rantisi']
name_inserted before: ['Abdel Aziz al-Rantisi']
name_inserted after: Abdel Aziz al-Rantisi
-----------------
name:  ['Abdel Aziz al-Rantisi']
native_name:  ['lang', 'ar', 'عبد العزيز الرنتيسي lang', 'Nobold', 'عبد العزيز الرنتيسي']
nickname:  -
religion:  -
party:  -
nationality:  -
Cannot translate phrase `Abdel Aziz alRantisi` to arabic
name ar:  -1
name ar:  -1
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Abraham Accords
en.wikipedia.org (imageinfo) File:President Trump and The First L...
en.wikipedia.org (parse) Abu Ali Mustafa
en.wikipedia.org (imageinfo) File:Abu ali mustafa ap.jpg


['lang', 'Abu Ali Mustafa', 'ar', 'أبو علي مصطفى']
['lang', 'Abu Ali Mustafa', 'ar', 'أبو علي مصطفى']
name before:  lang
name before:  Abu Ali Mustafa
name before:  ar
name before:  أبو علي مصطفى
name after:  Abu Ali Mustafa
-----------------
name:  ['lang', 'Abu Ali Mustafa', 'ar', 'أبو علي مصطفى']
native_name:  -
nickname:  -
religion:  -
party:  ['Popular Front for the Liberation of Palestine']
nationality:  ['Palestinian']
name ar:  أبو علي مصطفى 
START
political party:  Popular Front for the Liberation of Palestine
political party ar:  الجبهة الشعبية لتحرير فلسطين 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AC%D8%A8%D9%87%D8%A9_%D8%A7%D9%84%D8%B4%D8%B9%D8%A8%D9%8A%D8%A9_%D9%84%D8%AA%D8%AD%D8%B1%D9%8A%D8%B1_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86
END
START
nationality:  Palestinian
nationality ar:  فلسطينيون 
nationality ar link:  https://ar.wikipedia.org/wiki/%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D9%88%D9%86
END
name ar:  أبو علي مصطفى 
religion ar: 

en.wikipedia.org (parse) Abu Iyad
en.wikipedia.org (imageinfo) File:Salah Khalaf.jpg


...

['Yuval Diskin']
name_inserted before: ['Yuval Diskin']
name_inserted after: Yuval Diskin
-----------------
name:  ['Yuval Diskin']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  ['Israeli']
name ar:  يوفال ديسكين 
START
nationality:  Israeli
nationality ar:  إسرائيلي (مترجمه)
nationality ar link:  -1
END
name ar:  يوفال ديسكين 
religion ar:  -
party ar:  -
nationality ar:  ['إسرائيلي (مترجمه)']


en.wikipedia.org (parse) Zuheir Mohsen
en.wikipedia.org (imageinfo) File:Zuheir Mohsen.jpeg


['Zuheir Mohsen']
name_inserted before: ['Zuheir Mohsen']
name_inserted after: Zuheir Mohsen
-----------------
name:  ['Zuheir Mohsen']
native_name:  ['زهير محسن']
nickname:  -
religion:  -
party:  ['Baath party']
nationality:  ['Jordanian']
name ar:  زهير محسن 
START
political party:  Baath party
political party ar:  حزب البعث العربي الاشتراكي 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%A8%D8%B9%D8%AB_%D8%A7%D9%84%D8%B9%D8%B1%D8%A8%D9%8A_%D8%A7%D9%84%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A
END
START
nationality:  Jordanian
nationality ar:  أردني (مترجمه)
nationality ar link:  -1
END
name ar:  زهير محسن 
religion ar:  ['حزب البعث العربي الاشتراكي ']
party ar:  ['Baath party']
nationality ar:  ['أردني (مترجمه)']


en.wikipedia.org (parse) Zvi Zamir
en.wikipedia.org (imageinfo) File:Zvi Zamir 1979 (FL62867604).jpg


['Zvi Zamir']
name_inserted before: ['Zvi Zamir']
name_inserted after: Zvi Zamir
-----------------
name:  ['Zvi Zamir']
native_name:  ['צבי זמיר Script/Hebrew', 'Nobold', 'צבי זמיר', 'Script/Hebrew']
nickname:  -
religion:  -
party:  -
nationality:  ['Israeli']
name ar:  تسفي زامير 
START
nationality:  Israeli
nationality ar:  إسرائيلي (مترجمه)
nationality ar link:  -1
END
name ar:  تسفي زامير 
religion ar:  -
party ar:  -
nationality ar:  ['إسرائيلي (مترجمه)']
skipping Wikipedia:Citation needed
skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox 1982 Lebanon War
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data Soviet Union
skipping Template:Country data Syria
skipping Template:Country data United States
skipping Template:Israeli–Pal

In [ ]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Abdel Aziz al-Rantisi,-,-,-
1,Abu Ali Mustafa,Popular Front for the Liberation of Palestine,Palestinian,-
2,Salah Khalaf,Fatah,-,-
3,Khalil al-Wazir,-,-,-
4,Abu Nidal,,Palestinian,-
...,...,...,...,...
77,Yitzhak Rabin,"Alignment, Labor Party",Israeli,-
78,יצחק שמיר,,Israeli,-
79,Yuval Diskin,-,Israeli,-
80,Zuheir Mohsen,Baath party,Jordanian,-


In [ ]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,-,-,-1
1,-,فلسطينيون,الجبهة الشعبية لتحرير فلسطين,أبو علي مصطفى
2,-,-,حركة فتح,صلاح خلف
3,-,-,-,خليل الوزير
4,-,فلسطينيون,,صبري البنا
...,...,...,...,...
77,-,إسرائيلي (مترجمه),حزب العمال المحاذاة (مترجمه),إسحاق رابين
78,-,إسرائيلي (مترجمه),,إسحاق شامير
79,-,إسرائيلي (مترجمه),-,يوفال ديسكين
80,-,أردني (مترجمه),حزب البعث العربي الاشتراكي,زهير محسن


In [ ]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [ ]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)

                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'

                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p

                    df_locations_en = pd.concat([df_locations_en, pd.DataFrame({
                        'Name': [name],
                        'Subdivision type': [','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type],
                        'Subdivision name': [','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name]
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)


                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass

                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'


                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = pd.concat([df_locations_ar, pd.DataFrame({
                            'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                            'نوع التقسيم': [','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar],
                            'اسم التقسيم': [','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar],
                        })], ignore_index=True)

                    else:
                        print('skipping {} because name is not found'.format(p))

        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1920 Nebi Musa riots
en.wikipedia.org (imageinfo) File:Jerusalem-nabi-moussa-april-1920.jpg
en.wikipedia.org (parse) 1929 Hebron massacre
en.wikipedia.org (imageinfo) File:1929 Hebron massacre infobox.jpg
en.wikipedia.org (parse) 1929 Palestine riots
en.wikipedia.org (imageinfo) File:Jews flee the Old City of Jerus...
en.wikipedia.org (parse) 1936–1939 Arab revolt in Palestine
en.wikipedia.org (imageinfo) File:Train hostages.jpg
en.wikipedia.org (parse) 1947–1948 civil war in Mandatory Palestine
en.wikipedia.org (imageinfo) File:PikiWiki Israel 20804 The Palmach.jpg
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (parse) 1948 Palestine war
en.wikipedia.org (parse) 1948 Palestinian expulsion and flight
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1956 Rafah massacre
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation L

-----------------
name:  Algeria
subdivision_type:  -
subdivision_name:  -
name ar:  الجزائر 


en.wikipedia.org (parse) Ali Hassan Salameh
en.wikipedia.org (imageinfo) File:AliHassanSalameh.jpg
en.wikipedia.org (parse) All-Palestine Government
en.wikipedia.org (parse) All-Palestine Protectorate


-----------------
name:  Palestine
subdivision_type:  -
subdivision_name:  -
name ar:  دولة فلسطين 


en.wikipedia.org (parse) Amal Movement
en.wikipedia.org (parse) Ambassador
en.wikipedia.org (parse) American trusteeship proposal for Palestine
en.wikipedia.org (parse) Ami Ayalon
en.wikipedia.org (imageinfo) File:AmiAyalon.jpg
en.wikipedia.org (parse) Amin al-Husseini
en.wikipedia.org (imageinfo) File:Al-Husayni1929head.jpg
en.wikipedia.org (parse) Amnon Lipkin-Shahak
en.wikipedia.org (imageinfo) File:Amnon Lipkin-Shahak, Chief of G...
en.wikipedia.org (parse) Anglo-French Declaration
en.wikipedia.org (parse) Annapolis Conference
en.wikipedia.org (parse) Anti-Palestinianism
en.wikipedia.org (parse) Arab Higher Committee
en.wikipedia.org (parse) Arab League
en.wikipedia.org (parse) Arab Liberation Army
en.wikipedia.org (parse) Arab Liberation Front
en.wikipedia.org (parse) Arab Socialist Ba'ath Party – Lebanon Region
en.wikipedia.org (parse) Arab countries
en.wikipedia.org (imageinfo) File:Arab World (orthographic projec...
en.wikipedia.org (parse) Arab–Israeli conflict
en.wikipedia.or

...

-----------------
name:  Beirut
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon']
name ar:  بيروت 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) White Paper of 1939
en.wikipedia.org (imageinfo) File:1939 White Paper cmd 6019.djvu
en.wikipedia.org (parse) Wimpy Operation
en.wikipedia.org (parse) World Trade Center (1973–2001)
en.wikipedia.org (imageinfo) File:World Trade Center, New York Ci...
en.wikipedia.org (parse) Wye River Memorandum
en.wikipedia.org (parse) Yaakov Peri
en.wikipedia.org (parse) Yahya Ayyash
en.wikipedia.org (imageinfo) File:Yahya Ayyash.jpg
en.wikipedia.org (parse) Yasser Arafat
en.wikipedia.org (imageinfo) File:Leader of the PLO, Yasser Arafa...
en.wikipedia.org (parse) Yassir Arafat
en.wikipedia.org (imageinfo) File:Leader of the PLO, Yasser Arafa...
en.wikipedia.org (parse) Yemen


-----------------
name:  Yemen
subdivision_type:  -
subdivision_name:  -
name ar:  اليمن 


en.wikipedia.org (parse) Yitzhak Mordechai
en.wikipedia.org (imageinfo) File:Yitzhak Mordechai (cropped).jpg
en.wikipedia.org (parse) Yitzhak Rabin
en.wikipedia.org (imageinfo) File:Flickr - Israel Defense Forces ...
en.wikipedia.org (parse) Yitzhak Shamir
en.wikipedia.org (imageinfo) File:Yitzhak Shamir 1992 Dan Hadani ...
en.wikipedia.org (parse) Yom Kippur War
en.wikipedia.org (imageinfo) File:Yom Kippur War Montage.png
en.wikipedia.org (parse) Yuval Diskin
en.wikipedia.org (imageinfo) File:Yuval Diskin .jpg
en.wikipedia.org (parse) Zuheir Mohsen
en.wikipedia.org (imageinfo) File:Zuheir Mohsen.jpeg
en.wikipedia.org (parse) Zvi Zamir
en.wikipedia.org (imageinfo) File:Zvi Zamir 1979 (FL62867604).jpg


skipping Wikipedia:Citation needed
skipping Wikipedia:Protection policy
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Campaignbox 1982 Lebanon War
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data Soviet Union
skipping Template:Country data Syria
skipping Template:Country data United States
skipping Template:Israeli–Palestinian conflict
skipping Template talk:Arab–Israeli armed engagements
skipping Template talk:Arab–Israeli conflict
skipping Template talk:Arab–Israeli conflict participants
skipping Template talk:Arab–Israeli diplomacy
skipping Template talk:Campaignbox 1982 Lebanon War
skipping Template talk:Israeli–Palestinian conflict
skipping Category:Articles with unsourced statements from April 2023
skipping Category:Articles with unsourced statements from August 2011
sk

In [ ]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,Algeria,-,-
1,Palestine,-,-
2,Iraq,-,-
3,Beirut,Country,Lebanon
4,-,Country,Syria
5,Egypt,-,-
6,European Union,-,-
7,France,-,-
8,Greece,-,-
9,Iraq,-,-


In [ ]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,-,-,الجزائر
1,-,-,دولة فلسطين
2,-,-,العراق
3,لبنان,بلد,بيروت
4,-,-,مصر
5,-,-,الاتحاد الأوروبي
6,-,-,فرنسا
7,-,-,اليونان
8,-,-,العراق
9,-,-,إسرائيل


In [ ]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')